In [4]:
from os.path import join, isfile
from os import listdir
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
from scipy.spatial import distance

In [5]:
GENERAL_DIR = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien"
syn_list = ['ADNP', 'ANKRD11','CDK13', 'DEAF1', 'DYRK1A', 'EHMT1', 'FBXO11', 'SON', 'WAC', 'YY1', 'KDVS']

In [6]:
LEFT = [0, 1, 2, 3, 4, 5, 6, 7, 8, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341]
RIGHT = [9, 10, 11, 12, 13, 14, 15, 16, 17, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 157, 158, 168, 173, 174, 190, 191, 205, 210, 213, 217, 262, 263, 269, 278, 281, 282, 284, 288, 292, 294, 295, 296, 297, 298, 299, 302, 304, 305, 306, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509]

In [7]:
print(len(LEFT))

270


In [8]:
def load_data(syn, GENERAL_DIR):
    # files with representations
    syn_csv = GENERAL_DIR+ "\\features_facereader_landmarks_patient_groups.csv"
    ID_csv = GENERAL_DIR+ "\\features_facereader_landmarks_all_controls.csv"
    
    # open directories
    syn_dir = GENERAL_DIR+"\\{}\\{}-patients".format(syn, syn)
    ID_dir = GENERAL_DIR+ "\\{}\\{}-selected-ID-controls".format(syn, syn)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f))) and ".jpg" in f]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f))) and ".jpg" in f]
    
    data, labels, data_syn, data_ID = [], [], [], []
    
    with open (syn_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_syn: 
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append((float(row[i]), float(row[i+1]), float(row[i+2])))
                    i+=3    
                data_syn.append(rep)

                    
    with open (ID_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_ID: 
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append((float(row[i]), float(row[i+1]), float(row[i+2])))
                    i+=3    
                data_ID.append(rep)
    
    for index, (syn_item, ID_item) in enumerate(zip(data_syn, data_ID)):
        if len(syn_item) == 510 and len(ID_item) == 510: # hardcoded for now
            data.append(np.array(syn_item))
            labels.append(1)
            data.append(np.array(ID_item))
            labels.append(0)

    return np.array(data), np.array(labels)

In [9]:
def get_avg_dist(data, labels):
    dist_left_patient, dist_right_patient = [], []
    dist_left_control, dist_right_control = [], []
    symmetry_patients, symmetry_controls = [], []
    
    for i, row in enumerate(data):
        dist_left_points_patient, dist_right_points_patient = [], []
        dist_left_points_control, dist_right_points_control = [], []
            
        if labels[i] == 1:
            for p, _ in enumerate(row):
                if p in LEFT:
                    dist_left_points_patient.append(distance.euclidean(row[p], row[295]))
                if p in RIGHT:
                    dist_right_points_patient.append(distance.euclidean(row[p], row[295]))
                
            dist_left_patient.append(np.mean(dist_left_points_patient))        
            dist_right_patient.append(np.mean(dist_right_points_patient))   
            symmetry_patients.append(abs(np.mean(dist_left_points_patient) - np.mean(dist_right_points_patient)))

        else:
            for p, _ in enumerate(row):
                if p in LEFT:
                    dist_left_points_control.append(distance.euclidean(row[p], row[295]))
                if p in RIGHT:
                    dist_right_points_control.append(distance.euclidean(row[p], row[295]))

            dist_left_control.append(np.mean(dist_left_points_control))        
            dist_right_control.append(np.mean(dist_right_points_control))
            symmetry_controls.append(abs(np.mean(dist_left_points_control) - np.mean(dist_right_points_control)))
        
    stats = [np.mean(dist_left_patient), np.var(dist_left_patient), np.mean(dist_right_patient), np.var(dist_right_patient), np.mean(dist_left_control), np.var(dist_left_control), np.mean(dist_right_control), np.var(dist_right_control)]
    return stats, [symmetry_patients, symmetry_controls]

In [11]:
for syn in syn_list:   
    data, labels = load_data(syn, GENERAL_DIR)
    stats, symmetry = get_avg_dist(data, labels)
    
    print("Syndrome {}".format(syn))
    print("Patients")
    #print("Avg dist left:  {:.3f}, var: {:.3f} \nAvg dist right: {:.3f}, var: {:.3f}".format(stats[0], stats[1], stats[2], stats[3]))
    print("Symmetry score: {:.2f} with var :{:.2f}".format(np.mean(symmetry[0]), np.var(symmetry[0])))
    print("Controls")
    #print("Avg dist left:  {:.3f}, var: {:.3f} \nAvg dist right: {:.3f}, var: {:.3f}\n".format(stats[4], stats[5], stats[6], stats[7]))
    print("Symmetry score: {:.2f} with var: {:.2f}".format(np.mean(symmetry[1]), np.var(symmetry[1])))


Syndrome ADNP
Patients
Symmetry score: 1.92 with var :0.73
Controls
Symmetry score: 1.23 with var: 0.45
Syndrome ANKRD11
Patients
Symmetry score: 0.82 with var :0.42
Controls
Symmetry score: 0.99 with var: 0.60
Syndrome CDK13
Patients
Symmetry score: 1.23 with var :0.92
Controls
Symmetry score: 0.80 with var: 0.18
Syndrome DEAF1
Patients
Symmetry score: 0.86 with var :0.49
Controls
Symmetry score: 0.76 with var: 0.38
Syndrome DYRK1A
Patients
Symmetry score: 1.47 with var :1.99
Controls
Symmetry score: 0.92 with var: 0.63
Syndrome EHMT1
Patients
Symmetry score: 1.21 with var :1.60
Controls
Symmetry score: 0.90 with var: 1.09
Syndrome FBXO11
Patients
Symmetry score: 0.82 with var :0.33
Controls
Symmetry score: 0.94 with var: 0.40
Syndrome SON
Patients
Symmetry score: 0.91 with var :0.36
Controls
Symmetry score: 1.14 with var: 0.54
Syndrome WAC
Patients
Symmetry score: 1.26 with var :0.87
Controls
Symmetry score: 0.87 with var: 0.61
Syndrome YY1
Patients
Symmetry score: 1.30 with var :0.4